<a href="https://colab.research.google.com/github/a-t-em/nlp_logical_contradiction_classification/blob/main/nlp_logical_contradiction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 40.1 MB/s eta 0:00:00


In [2]:
pip install tensorflow

In [4]:
from transformers import BertTokenizer, TFBertModel, TFBertForSequenceClassification, RobertaTokenizer, TFRobertaForSequenceClassification,AutoTokenizer, TFAutoModelForSequenceClassification, TFAutoModel
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import layers
import sklearn
import pandas as pd
import io
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [5]:
from google.colab import files
uploaded_train = files.upload()

Saving train.csv to train.csv


In [6]:
from google.colab import files
uploaded_test = files.upload()

Saving test.csv to test.csv


EDA

In [7]:
# Loading the dataset
df = pd.read_csv(io.BytesIO(uploaded_train['train.csv']))
df_test = pd.read_csv(io.BytesIO(uploaded_test['test.csv']))

In [8]:
# Printing the first 5 rows of the DataFrame
df.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


In [9]:
df_test.head()

,id,premise,hypothesis,lang_abv,language
0,c6d58c3f69,بکس، کیسی، راہیل، یسعیاہ، کیلی، کیلی، اور کولم...,"کیسی کے لئے کوئی یادگار نہیں ہوگا, کولمین ہائی...",ur,Urdu
1,cefcc82292,هذا هو ما تم نصحنا به.,عندما يتم إخبارهم بما يجب عليهم فعله ، فشلت ال...,ar,Arabic
2,e98005252c,et cela est en grande partie dû au fait que le...,Les mères se droguent.,fr,French
3,58518c10ba,与城市及其他公民及社区组织代表就IMA的艺术发展进行对话&amp,IMA与其他组织合作，因为它们都依靠共享资金。,zh,Chinese
4,c32b0d16df,Она все еще была там.,"Мы думали, что она ушла, однако, она осталась.",ru,Russian


In [10]:
import plotly.express as px

labels, frequencies = np.unique(df.language.values, return_counts = True)

fig = px.pie(values=frequencies,
             names=labels,
             title='Languages distribution',
             color_discrete_sequence=px.colors.sequential.Plotly3)

fig.show()

# Based on the above chart, we can clearly see that there are a total of 15 languages of which 56.7% is from English language and the remaining 42.3% is divided among rest of the 14 languages.

In [11]:
import plotly.graph_objects as go

df['text_length'] = df['premise'].apply(len)

fig = go.Figure(data=[go.Histogram(x=df['text_length'],
                                   nbinsx=50,
                                   marker_color='skyblue')])
fig.update_layout(title_text='Text length distribution in "premise"', # title of plot
                  xaxis_title_text='Len of text in premise', # xaxis label
                  yaxis_title_text='Number of sentences', # yaxis label
                  bargap=0.2, # gap between bars of adjacent location coordinates
                  bargroupgap=0.1) # gap between bars of the same location coordinates
fig.show()


# There are more than 1500 sentences with less than 100 length of text in premise

In [12]:
df['text_length'] = df['hypothesis'].apply(len)

fig = go.Figure(data=[go.Histogram(x=df['text_length'],
                                   nbinsx=50,
                                   marker_color='skyblue')])
fig.update_layout(title_text='Text length distribution in "hypothesis"', # title of plot
                  xaxis_title_text='Len of text in hypothesis', # xaxis label
                  yaxis_title_text='Number of sentences', # yaxis label
                  bargap=0.2, # gap between bars of adjacent location coordinates
                  bargroupgap=0.1) # gap between bars of the same location coordinates
fig.show()

# The above graph shows that there are more than 2000 sentences with Length of text 50 in hypothesis

In [13]:
label_count = df['label'].value_counts().sort_index()
label_names = ['entailment', 'neutral', 'contradiction']
label_count.index = label_names

fig = go.Figure([go.Bar(x=label_names, y=label_count, marker_color='skyblue')])

fig.update_layout(title_text='Number of entries per label', # title of plot
                  xaxis_title_text='Label', # xaxis label
                  yaxis_title_text='Count', # yaxis label
                  )
fig.show()


# Based on the above chart, we can see clearly that the dataset is well balanced

# RoBERTa-based Sequence Classification for Natural Language Inference

In [14]:
# Preparing the dataset
X = df[['premise', 'hypothesis']].values
y = df['label'].values

In [15]:
# Initializing the RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [16]:
# Splitting the dataset into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
# Tokenizing and encoding the data
def get_encodings(data):
    premises, hypotheses = data[:, 0], data[:, 1]
    encodings = tokenizer(premises.tolist(), hypotheses.tolist(), padding='max_length', truncation=True, max_length=128, return_tensors='tf')
    inputs = {key: tf.constant(val) for key, val in encodings.items()}
    return inputs

In [18]:
train_inputs = get_encodings(X_train)
val_inputs = get_encodings(X_test)

# Preparing the input data
train_labels = tf.constant(y_train)
val_labels = tf.constant(y_test)

# Here we are creating a custom callback for early stopping
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [19]:
# Initializing the RoBERTa model
model = TFRobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [20]:
# Compiling the model
optimizer = keras.optimizers.Adam(learning_rate=2e-5)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [21]:
# Training the model with early stopping
history = model.fit(train_inputs, train_labels, epochs=20, batch_size=32, validation_data=(val_inputs, val_labels), callbacks=[early_stopping])

Epoch 1/20
303/303 [==============================] - 109s 237ms/step - loss: 1.0230 - accuracy: 0.4322 - val_loss: 0.8502 - val_accuracy: 0.5875
Epoch 2/20
303/303 [==============================] - 65s 214ms/step - loss: 0.7804 - accuracy: 0.6156 - val_loss: 0.7830 - val_accuracy: 0.6304
Epoch 3/20
303/303 [==============================] - 64s 211ms/step - loss: 0.6352 - accuracy: 0.6956 - val_loss: 0.7890 - val_accuracy: 0.6304
Epoch 4/20
303/303 [==============================] - 63s 209ms/step - loss: 0.5383 - accuracy: 0.7401 - val_loss: 0.8666 - val_accuracy: 0.6411
Epoch 5/20
303/303 [==============================] - 63s 210ms/step - loss: 0.4769 - accuracy: 0.7676 - val_loss: 0.9741 - val_accuracy: 0.6308


In [22]:
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(val_inputs, val_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

76/76 [==============================] - 6s 74ms/step - loss: 0.7830 - accuracy: 0.6304
Test Loss: 0.7829666137695312
Test Accuracy: 0.6303630471229553


# Multilingual BERT (mBERT) Sequence Classification for Natural Language Inference with Optimization Techniques (EarlyStopping to prevent overfitting)

In [23]:
# Initializing the mBERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [24]:
# Tokenizing and encoding the data
def get_encodings(data):
    premises, hypotheses = data[:, 0], data[:, 1]
    encodings = tokenizer(premises.tolist(), hypotheses.tolist(), padding='max_length', truncation=True, max_length=128, return_tensors='tf')
    inputs = {key: tf.constant(val) for key, val in encodings.items()}
    return inputs

In [ ]:
train_inputs = get_encodings(X_train)
val_inputs = get_encodings(X_test)

# Preparing the input data
train_labels = tf.constant(y_train)
val_labels = tf.constant(y_test)

In [27]:
# Compiling the model
optimizer = keras.optimizers.Adam(learning_rate=2e-5)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [29]:
# Training the model with early stopping
history = model.fit(train_inputs, train_labels, epochs=20, batch_size=32, validation_data=(val_inputs, val_labels), callbacks=[early_stopping])

Epoch 1/20
303/303 [==============================] - 105s 239ms/step - loss: 0.9327 - accuracy: 0.5549 - val_loss: 0.8264 - val_accuracy: 0.6258
Epoch 2/20
303/303 [==============================] - 66s 217ms/step - loss: 0.7013 - accuracy: 0.7078 - val_loss: 0.8204 - val_accuracy: 0.6572
Epoch 3/20
303/303 [==============================] - 65s 214ms/step - loss: 0.4904 - accuracy: 0.8098 - val_loss: 0.9284 - val_accuracy: 0.6448
Epoch 4/20
303/303 [==============================] - 64s 211ms/step - loss: 0.3146 - accuracy: 0.8852 - val_loss: 1.0922 - val_accuracy: 0.6427
Epoch 5/20
303/303 [==============================] - 64s 212ms/step - loss: 0.1947 - accuracy: 0.9322 - val_loss: 1.3186 - val_accuracy: 0.6361


In [30]:
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(val_inputs, val_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


76/76 [==============================] - 6s 75ms/step - loss: 0.8204 - accuracy: 0.6572
Test Loss: 0.8204382061958313
Test Accuracy: 0.6571782231330872


# XLM-RoBERTa Sequence Classification for Natural Language Inference with Enhanced Training Techniques

In [45]:

X = df[['premise', 'hypothesis']].values
y = df['label'].values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [46]:
train_inputs = get_encodings(X_train)
val_inputs = get_encodings(X_test)


train_labels = tf.constant(y_train)
val_labels = tf.constant(y_test)

In [47]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = TFAutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=3)

All PyTorch model weights were used when initializing TFXLMRobertaForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFXLMRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [48]:
def get_encodings(data):
    premises, hypotheses = data[:, 0], data[:, 1]
    encodings = tokenizer(premises.tolist(), hypotheses.tolist(), padding='max_length', truncation=True, max_length=128, return_tensors='tf')
    inputs = {key: tf.constant(val) for key, val in encodings.items()}
    return inputs

In [49]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

In [50]:
# Compiling the model
optimizer = keras.optimizers.Adam(learning_rate=2e-5)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [51]:
# Training the model with early stopping
history = model.fit(train_inputs, train_labels, epochs=20, batch_size=32, validation_data=(val_inputs, val_labels), callbacks=[early_stopping])

Epoch 1/20
303/303 [==============================] - 109s 256ms/step - loss: 1.0991 - accuracy: 0.3598 - val_loss: 1.0799 - val_accuracy: 0.4328
Epoch 2/20
303/303 [==============================] - 67s 222ms/step - loss: 1.0731 - accuracy: 0.4027 - val_loss: 1.0534 - val_accuracy: 0.4191
Epoch 3/20
303/303 [==============================] - 66s 218ms/step - loss: 0.9997 - accuracy: 0.4827 - val_loss: 0.8586 - val_accuracy: 0.6229
Epoch 4/20
303/303 [==============================] - 66s 217ms/step - loss: 0.8174 - accuracy: 0.6406 - val_loss: 0.7303 - val_accuracy: 0.6881
Epoch 5/20
303/303 [==============================] - 65s 214ms/step - loss: 0.6261 - accuracy: 0.7445 - val_loss: 0.7623 - val_accuracy: 0.6935
Epoch 6/20
303/303 [==============================] - 65s 214ms/step - loss: 0.4605 - accuracy: 0.8194 - val_loss: 0.7627 - val_accuracy: 0.7096
Epoch 7/20
303/303 [==============================] - 65s 213ms/step - loss: 0.3210 - accuracy: 0.8818 - val_loss: 0.8960 - val_a

In [52]:
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(val_inputs, val_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

76/76 [==============================] - 6s 75ms/step - loss: 0.7303 - accuracy: 0.6881
Test Loss: 0.7302846908569336
Test Accuracy: 0.6881188154220581


# Fine-Tuning XLM-RoBERTa for Natural Language Inference with Custom Architecture and Dropout Regularization

In [85]:
X = df[['premise', 'hypothesis']].values
y = df['label'].values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [86]:
train_inputs = get_encodings(X_train)
val_inputs = get_encodings(X_test)
train_labels = tf.constant(y_train)
val_labels = tf.constant(y_test)

In [87]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# Encoding function
def get_encodings(data):
    premises, hypotheses = data[:, 0], data[:, 1]
    encodings = tokenizer(premises.tolist(), hypotheses.tolist(), padding='max_length', truncation=True, max_length=128, return_tensors='tf')
    inputs = {key: tf.constant(val) for key, val in encodings.items()}
    return inputs

In [88]:
def create_model():
    base_model = TFAutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=3)
    input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="attention_mask")

    outputs = base_model([input_ids, attention_mask]).logits
    dropout = tf.keras.layers.Dropout(0.3)(outputs)  # adding dropout for regularization
    outputs = tf.keras.layers.Dense(3, activation='softmax')(dropout)

    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=outputs)
    return model

model = create_model()


early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


All PyTorch model weights were used when initializing TFXLMRobertaForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFXLMRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [89]:
# Compiling the model
optimizer = keras.optimizers.Adam(learning_rate=2e-5)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [90]:
# Callback for early stopping
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [91]:
history = model.fit(train_inputs, train_labels, epochs=20, batch_size=32, validation_data=(val_inputs, val_labels), callbacks=[early_stopping])

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5729: UserWarning:

"`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?



303/303 [==============================] - 110s 260ms/step - loss: 1.0930 - accuracy: 0.3583 - val_loss: 0.9933 - val_accuracy: 0.5384
Epoch 2/20
303/303 [==============================] - 67s 222ms/step - loss: 0.9637 - accuracy: 0.5209 - val_loss: 0.8176 - val_accuracy: 0.6432
Epoch 3/20
303/303 [==============================] - 66s 219ms/step - loss: 0.7975 - accuracy: 0.6350 - val_loss: 0.7304 - val_accuracy: 0.6885
Epoch 4/20
303/303 [==============================] - 66s 216ms/step - loss: 0.6845 - accuracy: 0.6880 - val_loss: 0.7226 - val_accuracy: 0.6955
Epoch 5/20
303/303 [==============================] - 65s 213ms/step - loss: 0.5677 - accuracy: 0.7397 - val_loss: 0.8372 - val_accuracy: 0.6951
Epoch 6/20
303/303 [==============================] - 65s 213ms/step - loss: 0.4887 - accuracy: 0.7706 - val_loss: 0.8865 - val_accuracy: 0.7162
Epoch 7/20
303/303 [==============================] - 65s 214ms/step - loss: 0.4307 - accuracy: 0.7874 - val_loss: 1.0531 - val_accuracy: 0.

In [92]:
test_loss, test_accuracy = model.evaluate(val_inputs, val_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

76/76 [==============================] - 6s 75ms/step - loss: 0.7226 - accuracy: 0.6955
Test Loss: 0.722647488117218
Test Accuracy: 0.6955445408821106


# Layer-wise Fine-Tuning of XLM-RoBERTa for Natural Language Inference with Learning Rate Scheduling

In [61]:
X = df[['premise', 'hypothesis']].values
y = df['label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [62]:
train_inputs = get_encodings(X_train)
val_inputs = get_encodings(X_test)
train_labels = tf.constant(y_train)
val_labels = tf.constant(y_test)

In [63]:
def create_model():

    base_model = TFAutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=3).roberta

    input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="attention_mask")

    embeddings = base_model([input_ids, attention_mask])
    pooled_output = embeddings[0][:, 0, :]
    dropout = tf.keras.layers.Dropout(0.3)(pooled_output)
    outputs = tf.keras.layers.Dense(3, activation='softmax')(dropout)

    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=outputs)
    return model


model = create_model()

All PyTorch model weights were used when initializing TFXLMRobertaForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFXLMRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [64]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Learning rate scheduler
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-5 * 10**(epoch / 20))

In [65]:
# Compiling the model
optimizer = keras.optimizers.Adam(learning_rate=1e-5)
loss = keras.losses.SparseCategoricalCrossentropy()
metric = keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [66]:
history = model.fit(train_inputs, train_labels, epochs=20, batch_size=32, validation_data=(val_inputs, val_labels), callbacks=[early_stopping, lr_schedule])

# After a few epochs, here we are unfreezing the last few layers of RoBERTa and continue training.
for layer in model.layers[-5:]:
    layer.trainable = True

optimizer = keras.optimizers.Adam(learning_rate=5e-6)
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

history_finetune = model.fit(train_inputs, train_labels, epochs=20, batch_size=32, validation_data=(val_inputs, val_labels), callbacks=[early_stopping])

Epoch 1/20
303/303 [==============================] - 110s 258ms/step - loss: 1.2769 - accuracy: 0.3861 - val_loss: 0.9335 - val_accuracy: 0.5726 - lr: 1.0000e-05
Epoch 2/20
303/303 [==============================] - 67s 221ms/step - loss: 0.9097 - accuracy: 0.6050 - val_loss: 0.7273 - val_accuracy: 0.6943 - lr: 1.1220e-05
Epoch 3/20
303/303 [==============================] - 66s 216ms/step - loss: 0.7216 - accuracy: 0.7102 - val_loss: 0.7276 - val_accuracy: 0.7054 - lr: 1.2589e-05
Epoch 4/20
303/303 [==============================] - 65s 213ms/step - loss: 0.5816 - accuracy: 0.7717 - val_loss: 0.7856 - val_accuracy: 0.7133 - lr: 1.4125e-05
Epoch 5/20
303/303 [==============================] - 65s 213ms/step - loss: 0.4508 - accuracy: 0.8278 - val_loss: 0.8211 - val_accuracy: 0.7046 - lr: 1.5849e-05
Epoch 6/20
303/303 [==============================] - 65s 214ms/step - loss: 0.3375 - accuracy: 0.8789 - val_loss: 0.8466 - val_accuracy: 0.7100 - lr: 1.7783e-05
Epoch 7/20
303/303 [=======

In [67]:
test_loss, test_accuracy = model.evaluate(val_inputs, val_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

76/76 [==============================] - 6s 75ms/step - loss: 0.7128 - accuracy: 0.7026
Test Loss: 0.7127985954284668
Test Accuracy: 0.7025577425956726


# Ensemble Learning for Natural Language Inference with Stacking and Averaging Strategies on XLM-RoBERTa

In [68]:
X = df[['premise', 'hypothesis']].values
y = df['label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [69]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# Encoding function
def get_encodings(data):
    premises, hypotheses = data[:, 0], data[:, 1]
    encodings = tokenizer(premises.tolist(), hypotheses.tolist(), padding='max_length', truncation=True, max_length=128, return_tensors='tf')
    inputs = {key: tf.constant(val) for key, val in encodings.items()}
    return inputs

train_inputs = get_encodings(X_train)
val_inputs = get_encodings(X_test)
train_labels = tf.constant(y_train)
val_labels = tf.constant(y_test)

In [70]:
def create_model():
    base_model = TFAutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=3).roberta
    input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="attention_mask")
    embeddings = base_model([input_ids, attention_mask])
    pooled_output = embeddings[0][:, 0, :]

    x = tf.keras.layers.BatchNormalization()(pooled_output)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    outputs = tf.keras.layers.Dense(3, activation='softmax')(x)

    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=outputs)
    return model

In [71]:
NUM_MODELS = 3
models = []
stacked_features_train = []
stacked_features_val = []

for i in range(NUM_MODELS):
    print(f"Training model {i+1}/{NUM_MODELS}...")
    model = create_model()
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-5 * 10**(epoch / 20))
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
    loss = tf.keras.losses.SparseCategoricalCrossentropy()
    metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
    model.compile(optimizer=optimizer, loss=loss, metrics=[metric])


    model.fit(train_inputs, train_labels, epochs=20, batch_size=32, validation_data=(val_inputs, val_labels), callbacks=[early_stopping, lr_schedule])

    train_preds = model.predict(train_inputs)
    val_preds = model.predict(val_inputs)
    stacked_features_train.append(train_preds)
    stacked_features_val.append(val_preds)


    models.append(model)
    del model
    tf.keras.backend.clear_session()

# Stacking
stacked_features_train = np.concatenate(stacked_features_train, axis=1)
stacked_features_val = np.concatenate(stacked_features_val, axis=1)

stacker = keras.Sequential([
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(3, activation='softmax')
])
stacker.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
stacker.fit(stacked_features_train, y_train, validation_data=(stacked_features_val, y_test), epochs=50, batch_size=32)

Training model 1/3...


All PyTorch model weights were used when initializing TFXLMRobertaForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFXLMRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/20
303/303 [==============================] - 109s 258ms/step - loss: 1.2638 - accuracy: 0.3395 - val_loss: 1.0933 - val_accuracy: 0.3523 - lr: 1.0000e-05
Epoch 2/20
303/303 [==============================] - 67s 223ms/step - loss: 1.1594 - accuracy: 0.4001 - val_loss: 1.0007 - val_accuracy: 0.5289 - lr: 1.1220e-05
Epoch 3/20
303/303 [==============================] - 66s 219ms/step - loss: 1.0102 - accuracy: 0.5396 - val_loss: 0.8323 - val_accuracy: 0.6349 - lr: 1.2589e-05
Epoch 4/20
303/303 [==============================] - 65s 216ms/step - loss: 0.8445 - accuracy: 0.6446 - val_loss: 0.7379 - val_accuracy: 0.6877 - lr: 1.4125e-05
Epoch 5/20
303/303 [==============================] - 65s 214ms/step - loss: 0.7304 - accuracy: 0.7099 - val_loss: 0.7736 - val_accuracy: 0.6790 - lr: 1.5849e-05
Epoch 6/20
303/303 [==============================] - 65s 213ms/step - loss: 0.5972 - accuracy: 0.7762 - val_loss: 0.7836 - val_accuracy: 0.7050 - lr: 1.7783e-05
Epoch 7/20
303/303 [=======

All PyTorch model weights were used when initializing TFXLMRobertaForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFXLMRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/20
303/303 [==============================] - 110s 259ms/step - loss: 1.2622 - accuracy: 0.3363 - val_loss: 1.0973 - val_accuracy: 0.3366 - lr: 1.0000e-05
Epoch 2/20
303/303 [==============================] - 67s 221ms/step - loss: 1.1890 - accuracy: 0.3581 - val_loss: 1.0391 - val_accuracy: 0.4571 - lr: 1.1220e-05
Epoch 3/20
303/303 [==============================] - 66s 218ms/step - loss: 1.0323 - accuracy: 0.5142 - val_loss: 0.8397 - val_accuracy: 0.6324 - lr: 1.2589e-05
Epoch 4/20
303/303 [==============================] - 66s 217ms/step - loss: 0.8517 - accuracy: 0.6433 - val_loss: 0.7801 - val_accuracy: 0.6683 - lr: 1.4125e-05
Epoch 5/20
303/303 [==============================] - 65s 215ms/step - loss: 0.7133 - accuracy: 0.7192 - val_loss: 0.8132 - val_accuracy: 0.6790 - lr: 1.5849e-05
Epoch 6/20
303/303 [==============================] - 65s 214ms/step - loss: 0.5864 - accuracy: 0.7735 - val_loss: 0.8025 - val_accuracy: 0.6939 - lr: 1.7783e-05
Epoch 7/20
303/303 [=======

All PyTorch model weights were used when initializing TFXLMRobertaForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFXLMRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/20
303/303 [==============================] - 110s 258ms/step - loss: 1.2716 - accuracy: 0.3407 - val_loss: 1.0966 - val_accuracy: 0.3362 - lr: 1.0000e-05
Epoch 2/20
303/303 [==============================] - 67s 222ms/step - loss: 1.1621 - accuracy: 0.3905 - val_loss: 0.9821 - val_accuracy: 0.5421 - lr: 1.1220e-05
Epoch 3/20
303/303 [==============================] - 66s 218ms/step - loss: 0.9802 - accuracy: 0.5611 - val_loss: 0.8399 - val_accuracy: 0.6213 - lr: 1.2589e-05
Epoch 4/20
303/303 [==============================] - 66s 217ms/step - loss: 0.8370 - accuracy: 0.6525 - val_loss: 0.7405 - val_accuracy: 0.6943 - lr: 1.4125e-05
Epoch 5/20
303/303 [==============================] - 65s 216ms/step - loss: 0.7234 - accuracy: 0.7178 - val_loss: 0.7123 - val_accuracy: 0.7046 - lr: 1.5849e-05
Epoch 6/20
303/303 [==============================] - 65s 214ms/step - loss: 0.6148 - accuracy: 0.7697 - val_loss: 0.7448 - val_accuracy: 0.6902 - lr: 1.7783e-05
Epoch 7/20
303/303 [=======

In [72]:
# Ensemble prediction
def ensemble_predict(models, inputs):
    predictions = [model.predict(inputs) for model in models]
    avg_prediction = np.mean(predictions, axis=0)
    return np.argmax(avg_prediction, axis=1)

ensemble_preds = ensemble_predict(models, val_inputs)
accuracy = accuracy_score(y_test, ensemble_preds)
print(f"Ensemble Test Accuracy: {accuracy * 100:.2f}%")

76/76 [==============================] - 6s 72ms/step
Ensemble Test Accuracy: 70.50%


# Ensemble Learning for Natural Language Inference with Stacking, Averaging, and L2 Regularization on XLM-RoBERTa with Bidirectional LSTM

In [73]:
X = df[['premise', 'hypothesis']].values
y = df['label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [74]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

def get_encodings(data):
    premises, hypotheses = data[:, 0], data[:, 1]
    encodings = tokenizer(premises.tolist(), hypotheses.tolist(), padding='max_length', truncation=True, max_length=128, return_tensors='tf')
    inputs = {key: tf.constant(val) for key, val in encodings.items()}
    return inputs

train_inputs = get_encodings(X_train)
val_inputs = get_encodings(X_test)
train_labels = tf.constant(y_train)
val_labels = tf.constant(y_test)

In [75]:
def create_model():
    base_model = TFAutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=3).roberta
    input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="attention_mask")
    embeddings = base_model([input_ids, attention_mask])
    sequence_output = embeddings[0]

    # Adding a Bidirectional LSTM layer
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(sequence_output)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    x = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    outputs = tf.keras.layers.Dense(3, activation='softmax')(x)

    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=outputs)
    return model

In [76]:
NUM_MODELS = 3
models = []
stacked_features_train = []
stacked_features_val = []

for i in range(NUM_MODELS):
    print(f"Training model {i+1}/{NUM_MODELS}...")
    model = create_model()


    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
    lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-5 * 10**(epoch / 20))

    optimizer = keras.optimizers.Adam(learning_rate=1e-5)
    loss = keras.losses.SparseCategoricalCrossentropy()
    metric = keras.metrics.SparseCategoricalAccuracy('accuracy')
    model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

    model.fit(train_inputs, train_labels, epochs=20, batch_size=32, validation_data=(val_inputs, val_labels), callbacks=[early_stopping, reduce_lr, lr_schedule])

    train_preds = model.predict(train_inputs)
    val_preds = model.predict(val_inputs)

    stacked_features_train.append(train_preds)
    stacked_features_val.append(val_preds)

    models.append(model)
    tf.keras.backend.clear_session()

# Stacking
stacked_features_train = np.concatenate(stacked_features_train, axis=1)
stacked_features_val = np.concatenate(stacked_features_val, axis=1)

stacker = keras.Sequential([
    keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(3, activation='softmax')
])
stacker.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
stacker.fit(stacked_features_train, y_train, validation_data=(stacked_features_val, y_test), epochs=50, batch_size=32)

Training model 1/3...


All PyTorch model weights were used when initializing TFXLMRobertaForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFXLMRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/20
303/303 [==============================] - 115s 268ms/step - loss: 1.6617 - accuracy: 0.3340 - val_loss: 1.6395 - val_accuracy: 0.3424 - lr: 1.0000e-05
Epoch 2/20
303/303 [==============================] - 71s 233ms/step - loss: 1.6446 - accuracy: 0.3401 - val_loss: 1.6311 - val_accuracy: 0.3412 - lr: 1.1220e-05
Epoch 3/20
303/303 [==============================] - 70s 230ms/step - loss: 1.6232 - accuracy: 0.3741 - val_loss: 1.5675 - val_accuracy: 0.4686 - lr: 1.2589e-05
Epoch 4/20
303/303 [==============================] - 69s 228ms/step - loss: 1.5549 - accuracy: 0.4593 - val_loss: 1.4884 - val_accuracy: 0.5149 - lr: 1.4125e-05
Epoch 5/20
303/303 [==============================] - 69s 227ms/step - loss: 1.4270 - accuracy: 0.5834 - val_loss: 1.3372 - val_accuracy: 0.6452 - lr: 1.5849e-05
Epoch 6/20
303/303 [==============================] - 68s 226ms/step - loss: 1.2929 - accuracy: 0.6816 - val_loss: 1.2604 - val_accuracy: 0.6848 - lr: 1.7783e-05
Epoch 7/20
303/303 [=======

All PyTorch model weights were used when initializing TFXLMRobertaForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFXLMRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/20
303/303 [==============================] - 115s 271ms/step - loss: 1.6593 - accuracy: 0.3363 - val_loss: 1.6364 - val_accuracy: 0.3457 - lr: 1.0000e-05
Epoch 2/20
303/303 [==============================] - 71s 233ms/step - loss: 1.6460 - accuracy: 0.3354 - val_loss: 1.6328 - val_accuracy: 0.3511 - lr: 1.1220e-05
Epoch 3/20
303/303 [==============================] - 69s 229ms/step - loss: 1.6470 - accuracy: 0.3391 - val_loss: 1.6238 - val_accuracy: 0.3300 - lr: 1.2589e-05
Epoch 4/20
303/303 [==============================] - 69s 228ms/step - loss: 1.6313 - accuracy: 0.3345 - val_loss: 1.6164 - val_accuracy: 0.3300 - lr: 1.4125e-05
Epoch 5/20
303/303 [==============================] - 69s 228ms/step - loss: 1.6175 - accuracy: 0.3350 - val_loss: 1.6071 - val_accuracy: 0.3511 - lr: 1.5849e-05
Epoch 6/20
303/303 [==============================] - 69s 229ms/step - loss: 1.6077 - accuracy: 0.3295 - val_loss: 1.5970 - val_accuracy: 0.3556 - lr: 1.7783e-05
Epoch 7/20
303/303 [=======

All PyTorch model weights were used when initializing TFXLMRobertaForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFXLMRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/20
303/303 [==============================] - 116s 269ms/step - loss: 1.6590 - accuracy: 0.3356 - val_loss: 1.6375 - val_accuracy: 0.3230 - lr: 1.0000e-05
Epoch 2/20
303/303 [==============================] - 71s 233ms/step - loss: 1.6496 - accuracy: 0.3332 - val_loss: 1.6282 - val_accuracy: 0.3453 - lr: 1.1220e-05
Epoch 3/20
303/303 [==============================] - 70s 230ms/step - loss: 1.6324 - accuracy: 0.3380 - val_loss: 1.6218 - val_accuracy: 0.3337 - lr: 1.2589e-05
Epoch 4/20
303/303 [==============================] - 69s 228ms/step - loss: 1.6264 - accuracy: 0.3371 - val_loss: 1.6004 - val_accuracy: 0.4200 - lr: 1.4125e-05
Epoch 5/20
303/303 [==============================] - 69s 228ms/step - loss: 1.5626 - accuracy: 0.4426 - val_loss: 1.4118 - val_accuracy: 0.6155 - lr: 1.5849e-05
Epoch 6/20
303/303 [==============================] - 69s 227ms/step - loss: 1.3848 - accuracy: 0.6268 - val_loss: 1.2867 - val_accuracy: 0.6675 - lr: 1.7783e-05
Epoch 7/20
303/303 [=======

In [77]:
# Ensemble prediction using the stacker
stacked_preds = np.argmax(stacker.predict(stacked_features_val), axis=1)
accuracy = accuracy_score(y_test, stacked_preds)
print(f"Stacked Test Accuracy: {accuracy * 100:.2f}%")

76/76 [==============================] - 0s 1ms/step
Stacked Test Accuracy: 70.79%


In [78]:
# Ensemble prediction
def ensemble_predict(models, inputs):
    predictions = [model.predict(inputs) for model in models]
    avg_prediction = np.mean(predictions, axis=0)
    return np.argmax(avg_prediction, axis=1)

ensemble_preds = ensemble_predict(models, val_inputs)
accuracy = accuracy_score(y_test, ensemble_preds)
print(f"Ensemble Test Accuracy: {accuracy * 100:.2f}%")

76/76 [==============================] - 7s 76ms/step
Ensemble Test Accuracy: 70.87%


# Natural Language Inference using Ensemble of XLM-RoBERTa Trained on SNLI, MNLI, ANLI, and XNLI Datasets with Stacking and Averaging

In [79]:
X = df[['premise', 'hypothesis']].values
y = df['label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [80]:
tokenizer = AutoTokenizer.from_pretrained("symanto/xlm-roberta-base-snli-mnli-anli-xnli")

def get_encodings(data):
    premises, hypotheses = data[:, 0], data[:, 1]
    encodings = tokenizer(premises.tolist(), hypotheses.tolist(), padding='max_length', truncation=True, max_length=128, return_tensors='tf')
    inputs = {key: tf.constant(val) for key, val in encodings.items()}
    return inputs

train_inputs = get_encodings(X_train)
val_inputs = get_encodings(X_test)
train_labels = tf.constant(y_train)
val_labels = tf.constant(y_test)

In [81]:
def create_model():
    base_model = TFAutoModel.from_pretrained("symanto/xlm-roberta-base-snli-mnli-anli-xnli", from_pt=True)
    input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="attention_mask")
    embeddings = base_model([input_ids, attention_mask])
    pooled_output = embeddings[0][:, 0, :]

    x = tf.keras.layers.BatchNormalization()(pooled_output)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    outputs = tf.keras.layers.Dense(3, activation='softmax')(x)

    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=outputs)

    return model

In [82]:
NUM_MODELS = 3
models = []
stacked_features_train = []
stacked_features_val = []

for i in range(NUM_MODELS):
    print(f"Training model {i+1}/{NUM_MODELS}...")
    model = create_model()


    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
    lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-5 * 10**(epoch / 20))

    optimizer = keras.optimizers.Adam(learning_rate=1e-5)
    loss = keras.losses.SparseCategoricalCrossentropy()
    metric = keras.metrics.SparseCategoricalAccuracy('accuracy')
    model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

    model.fit(train_inputs, train_labels, epochs=20, batch_size=32, validation_data=(val_inputs, val_labels), callbacks=[early_stopping, reduce_lr, lr_schedule])

    train_preds = model.predict(train_inputs)
    val_preds = model.predict(val_inputs)

    stacked_features_train.append(train_preds)
    stacked_features_val.append(val_preds)

    models.append(model)
    tf.keras.backend.clear_session()

# Stacking
stacked_features_train = np.concatenate(stacked_features_train, axis=1)
stacked_features_val = np.concatenate(stacked_features_val, axis=1)

stacker = keras.Sequential([
    keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(3, activation='softmax')
])
stacker.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
stacker.fit(stacked_features_train, y_train, validation_data=(stacked_features_val, y_test), epochs=50, batch_size=32)

Training model 1/3...


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFXLMRobertaModel: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'roberta.embeddings.position_ids', 'classifier.out_proj.weight']
- This IS expected if you are initializing TFXLMRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFXLMRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and 

Epoch 1/20


303/303 [==============================] - 113s 257ms/step - loss: 0.4153 - accuracy: 0.8433 - val_loss: 0.2748 - val_accuracy: 0.8927 - lr: 1.0000e-05
Epoch 2/20
303/303 [==============================] - 67s 221ms/step - loss: 0.2767 - accuracy: 0.8999 - val_loss: 0.2939 - val_accuracy: 0.8915 - lr: 1.1220e-05
Epoch 3/20
303/303 [==============================] - 66s 218ms/step - loss: 0.2042 - accuracy: 0.9299 - val_loss: 0.3530 - val_accuracy: 0.8923 - lr: 1.2589e-05
Epoch 4/20
303/303 [==============================] - 65s 215ms/step - loss: 0.1768 - accuracy: 0.9402 - val_loss: 0.3499 - val_accuracy: 0.8919 - lr: 2.8251e-06
Epoch 5/20
303/303 [==============================] - 65s 216ms/step - loss: 0.1379 - accuracy: 0.9530 - val_loss: 0.4701 - val_accuracy: 0.8833 - lr: 1.5849e-05
Epoch 6/20
76/76 [==============================] - 9s 73ms/step
Training model 2/3...


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFXLMRobertaModel: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'roberta.embeddings.position_ids', 'classifier.out_proj.weight']
- This IS expected if you are initializing TFXLMRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFXLMRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and 

Epoch 1/20


303/303 [==============================] - 110s 259ms/step - loss: 0.3967 - accuracy: 0.8494 - val_loss: 0.2850 - val_accuracy: 0.8956 - lr: 1.0000e-05
Epoch 2/20
303/303 [==============================] - 66s 219ms/step - loss: 0.2684 - accuracy: 0.9050 - val_loss: 0.3085 - val_accuracy: 0.8940 - lr: 1.1220e-05
Epoch 3/20
303/303 [==============================] - 65s 216ms/step - loss: 0.2230 - accuracy: 0.9224 - val_loss: 0.3284 - val_accuracy: 0.8956 - lr: 1.2589e-05
Epoch 4/20
303/303 [==============================] - 65s 216ms/step - loss: 0.1588 - accuracy: 0.9461 - val_loss: 0.3956 - val_accuracy: 0.8742 - lr: 2.8251e-06
Epoch 5/20
303/303 [==============================] - 65s 215ms/step - loss: 0.1326 - accuracy: 0.9564 - val_loss: 0.4811 - val_accuracy: 0.8841 - lr: 1.5849e-05
Epoch 6/20
76/76 [==============================] - 9s 74ms/step
Training model 3/3...


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFXLMRobertaModel: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'roberta.embeddings.position_ids', 'classifier.out_proj.weight']
- This IS expected if you are initializing TFXLMRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFXLMRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and 

Epoch 1/20


303/303 [==============================] - 109s 258ms/step - loss: 0.3900 - accuracy: 0.8525 - val_loss: 0.2767 - val_accuracy: 0.8944 - lr: 1.0000e-05
Epoch 2/20
303/303 [==============================] - 67s 222ms/step - loss: 0.2745 - accuracy: 0.9032 - val_loss: 0.3089 - val_accuracy: 0.8948 - lr: 1.1220e-05
Epoch 3/20
303/303 [==============================] - 65s 216ms/step - loss: 0.2058 - accuracy: 0.9281 - val_loss: 0.3420 - val_accuracy: 0.8903 - lr: 1.2589e-05
Epoch 4/20
303/303 [==============================] - 65s 215ms/step - loss: 0.1735 - accuracy: 0.9395 - val_loss: 0.4212 - val_accuracy: 0.8812 - lr: 2.8251e-06
Epoch 5/20
303/303 [==============================] - 65s 215ms/step - loss: 0.1249 - accuracy: 0.9578 - val_loss: 0.4872 - val_accuracy: 0.8767 - lr: 1.5849e-05
Epoch 6/20
76/76 [==============================] - 8s 73ms/step
Epoch 1/50
303/303 [==============================] - 2s 3ms/step - loss: 0.4284 - accuracy: 0.8976 - val_loss: 0.3226 - val_accuracy: 

In [83]:
# Ensemble prediction using the stacker
stacked_preds = np.argmax(stacker.predict(stacked_features_val), axis=1)
accuracy = accuracy_score(y_test, stacked_preds)
print(f"Stacked Test Accuracy: {accuracy * 100:.2f}%")

76/76 [==============================] - 0s 1ms/step
Stacked Test Accuracy: 90.06%


In [94]:
# Ensemble prediction
def ensemble_predict(models, inputs):
    predictions = [model.predict(inputs) for model in models]
    avg_prediction = np.mean(predictions, axis=0)
    return np.argmax(avg_prediction, axis=1)

ensemble_preds = ensemble_predict(models, val_inputs)
accuracy = accuracy_score(y_test, ensemble_preds)
print(f"Ensemble Test Accuracy: {accuracy * 100:.2f}%")

76/76 [==============================] - 6s 74ms/step
Ensemble Test Accuracy: 89.77%


# So as the above model with stacker has the highest accuracy of 90.06%, we are considering that as our final model for prediction

# Predicting and Saving Test Results Using the Stacked Ensemble Model

In [93]:
X_test_data = df_test[['premise', 'hypothesis']].values
test_inputs = get_encodings(X_test_data)

# Predicting using each base model
stacked_features_test = []
for model in models:
    test_preds = model.predict(test_inputs)
    stacked_features_test.append(test_preds)

# Stacking
stacked_features_test = np.concatenate(stacked_features_test, axis=1)

# Predicting using the above stacker model
stacked_test_preds = np.argmax(stacker.predict(stacked_features_test), axis=1)

df_test['predictions'] = stacked_test_preds
df_test[['premise', 'hypothesis', 'predictions']].to_csv('test_predictions.csv', index=False)

163/163 [==============================] - 0s 1ms/step


In [95]:
df_test.head()

,id,premise,hypothesis,lang_abv,language,predictions
0,c6d58c3f69,بکس، کیسی، راہیل، یسعیاہ، کیلی، کیلی، اور کولم...,"کیسی کے لئے کوئی یادگار نہیں ہوگا, کولمین ہائی...",ur,Urdu,2
1,cefcc82292,هذا هو ما تم نصحنا به.,عندما يتم إخبارهم بما يجب عليهم فعله ، فشلت ال...,ar,Arabic,1
2,e98005252c,et cela est en grande partie dû au fait que le...,Les mères se droguent.,fr,French,0
3,58518c10ba,与城市及其他公民及社区组织代表就IMA的艺术发展进行对话&amp,IMA与其他组织合作，因为它们都依靠共享资金。,zh,Chinese,1
4,c32b0d16df,Она все еще была там.,"Мы думали, что она ушла, однако, она осталась.",ru,Russian,1


# Creating subset which is in accordance with the Kaggle submission csv file

In [97]:
stacked_test_preds = np.argmax(stacker.predict(stacked_features_test), axis=1)

# We are creating a subset dataframe with 'id' and 'predictions'
df_subset = pd.DataFrame({
    'id': df_test['id'],
    'prediction': stacked_test_preds
})

df_subset.to_csv('submission.csv', index=False)

163/163 [==============================] - 0s 1ms/step


In [98]:
df_sub = pd.read_csv('submission.csv')
df_sub.head()

,id,prediction
0,c6d58c3f69,2
1,cefcc82292,1
2,e98005252c,0
3,58518c10ba,1
4,c32b0d16df,1
